## Logistic Regression

- why need different function for classification?
- sigmoid

## Neural Network for Logistic Regression

- activation function

## Pytorch

## Adding nodes and layers

- types of functions with multiple nodes, layers (not just logistic regression anymore?)

## ReLU

- why? briefly, gradients


## Learning Arbitrary Functions - Intuition

- intuition around how this works/allows different non-linear functions to be learned (nodes turning on/off at different values...)